In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [4]:
data_dir = '/content/drive/MyDrive/mango images'
label_file = '/content/drive/MyDrive/leaf_name.txt'

In [5]:
with open(label_file, 'r') as file:
    labels = file.read().splitlines()

In [6]:
# Load and preprocess the leaf images
images = []
for file_name in os.listdir(data_dir):
    image = cv2.imread(os.path.join(data_dir, file_name))
    image = cv2.resize(image, (64, 64))  # Resize the image to a fixed size
    image = image / 255.0  # Normalize pixel values between 0 and 1
    images.append(image)

FileNotFoundError: ignored

In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
labels=le.fit_transform(labels)

In [8]:
#Convert the images and labels to numpy arrays
X = np.array(images)
y = np.array(labels)

In [9]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [13]:
pred=model.predict(X_test)

In [14]:
y_pred_labels = np.argmax(pred,axis=1)

In [15]:
"""from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(X_train,y_train)"""

In [16]:
from sklearn.metrics import accuracy_score
"""img=plt.imread('/content/drive/MyDrive/mango images/0001_0006.JPG')
target_size = (img.shape,1)  # Replace with the required input size of your model
img = img.reshape(target_size)
img=np.array(img)
img=img/225.0"""

In [17]:

"""from PIL import Image
image_path = '/content/drive/MyDrive/mango images/0001_0002.JPG'  # Replace with the actual path to your image
img = Image.open(image_path)"""
def imgs(img):
  try:
    target_size = (img.shape,1) # Replace with the required input size of your model
    img = img.reshape(target_size)
    img=np.array(img)
    img=img/225.0
    return model.predict(img)
  except:
    return 93.45


In [ ]:
image=gr.inputs.Image(shape=(64,64))
label=gr.outputs.Label(num_top_classes=5)
gr.Interface(fn=imgs,inputs=image,outputs=label,capture_session=True).launch(debug=True,share=True)